# **1. Install Lib**

In [ ]:
!pip install facebook-scraper

     |████████████████████████████████| 153kB 7.0MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13484 sha256=f7a97512c4d2dbb80acc9b4fd56f1129a38997e8632fabfed20218e3c6c3f433
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for parse: filename=parse-1.15.0-cp36-none-any.whl size=23710 sha256=3ac63a63d7087a091870048e708e8ea310bf5ba83e3e7b0f655421bcd026103a
  Stored in directory: /root/.cache/pip/wheels/07/ee/c8/eced0759f09fc635398ab1b8e89c38549b28e5db7fd4a53ba5
Successfully built fake-useragent parse
ERROR: pyppeteer 0.2.2 has requirement urllib3<2.0.0,>=1.25.8, but you'll have urllib3 1.24.3 which is incompatible.
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [ ]:
!pip install xlsxwriter

     |████████████████████████████████| 143kB 2.7MB/s 


# **2. Init Lib**

In [ ]:
from facebook_scraper import get_posts
import pandas as pd
import time

# **3. Define Helper Function**

In [ ]:
# function to display time more elegantly
intervals = (
    ('weeks', 604800),  # 60 * 60 * 24 * 7
    ('days', 86400),    # 60 * 60 * 24
    ('hours', 3600),    # 60 * 60
    ('minutes', 60),
    ('seconds', 1),
    )

def display_time(seconds, granularity=2):
    result = []

    for name, count in intervals:
        value = seconds // count
        if value:
            seconds -= value * count
            if value == 1:
                name = name.rstrip('s')
            result.append("{} {}".format(value, name))
    return ', '.join(result[:granularity])

In [ ]:
import re
from bs4 import BeautifulSoup
import lxml
from nltk.tokenize import WordPunctTokenizer

def text_cleaner(text):
  tok = WordPunctTokenizer()
  pat1 = r'@[A-Za-z0-9]+'
  pat2 = r'\w+:\/\/\S+'
  pat3 = r'\S*twitter.com\S*' 
  combined_pat = r'|'.join((pat1, pat2, pat3))
  soup = BeautifulSoup(text, 'lxml')
  souped = soup.get_text()
  stripped = re.sub(combined_pat, '', souped)
  try:
    clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
  except:
    clean = stripped
  letters_only = re.sub("[^a-zA-Z0-9]", " ", clean)
  lower_case = letters_only.lower()
  words = tok.tokenize(lower_case)
  return (" ".join(words)).strip()

# **4. Define Crawling Function**

In [ ]:
def generate_excel(generator, id, clean):
  data = []
  for post in generator:
    if post['text'] == '':
      pass
    else:
      data.append(post)
  df = pd.DataFrame(data)
  df.drop(['post_text'], axis=1, inplace=True)

  if clean:
    df['text'] = df['text'].apply(lambda text: text_cleaner(text))

  path = "/content/"
  writer = pd.ExcelWriter(f"{path}{id}.xlsx", engine = 'xlsxwriter')
  df.to_excel(writer, index=False)
  writer.save()
  writer.close()

  print(f"Crawling is complete! get: {df.shape[0]} datas")
  return df

In [ ]:
def get_group_post(group_id, pages=1, clean=False):
  """
  - param 'group_id' diisi dengan id group yang di dapat dari url group
    contoh https://www.facebook.com/groups/101812157281468
    maka id groupnya = 101812157281468

  - param 'pages' berisi angka, berapa halaman dari group tersebut yang
    ingin di crawling

  - param 'clean' secara default False, 
    set True untuk membersihkan hasil post
  """
  start_time = time.time()
  post_result = get_posts(group=group_id, pages=pages)
  
  generate_excel(post_result,group_id, clean)
  print("Exe Time : ", display_time(time.time() - start_time))

In [ ]:
def get_page_post(page_id, pages=1, clean=False):
  """
  - param 'page_id' diisi dengan id page yang didapat dari url page
    contoh https://www.facebook.com/BRIofficialpage/
    maka id pagenya = BRIofficialpage

  - param 'pages' berisi angka, berapa halaman dari page tersebut yang
    ingin di crawling

  - param 'clean' secara default False, 
    set True untuk membersihkan hasil post
  """

  start_time = time.time()
  post_result = get_posts(page_id, pages=pages)
  
  generate_excel(post_result, page_id, clean)
  print("Exe Time : ", display_time(time.time() - start_time))

# **5. Run Code!!!**

In [ ]:
df = get_group_post('101812157281468', 2, True)

Crawling is complete! get: 31 datas
Exe Time :  1.0 second


In [ ]:
df = get_page_post('BRIofficialpage', 2)

Crawling is complete! get: 6 datas
Exe Time :  1.0 second
